In [1]:
import pandas as pd
import numpy as np

In [2]:
adm = pd.read_csv("cleaned_admission.csv")

In [24]:
adm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58976 entries, 0 to 58975
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   SUBJECT_ID            58976 non-null  int64 
 1   HADM_ID               58976 non-null  int64 
 2   ADMITTIME             58976 non-null  object
 3   DISCHTIME             58976 non-null  object
 4   DEATHTIME             5854 non-null   object
 5   ADMISSION_TYPE        58976 non-null  object
 6   INSURANCE             58976 non-null  object
 7   MARITAL_STATUS        48848 non-null  object
 8   ETHNICITY             58976 non-null  object
 9   DIAGNOSIS             58951 non-null  object
 10  HOSPITAL_EXPIRE_FLAG  58976 non-null  int64 
 11  ETHNIC_GROUP          58976 non-null  object
dtypes: int64(3), object(9)
memory usage: 5.4+ MB


In [29]:
adm["ETHNICITY"].value_counts(dropna = False)

ETHNICITY
WHITE                                                       40996
BLACK/AFRICAN AMERICAN                                       5440
UNKNOWN/NOT SPECIFIED                                        4523
HISPANIC OR LATINO                                           1696
OTHER                                                        1512
ASIAN                                                        1509
UNABLE TO OBTAIN                                              814
PATIENT DECLINED TO ANSWER                                    559
ASIAN - CHINESE                                               277
HISPANIC/LATINO - PUERTO RICAN                                232
BLACK/CAPE VERDEAN                                            200
WHITE - RUSSIAN                                               164
MULTI RACE ETHNICITY                                          130
BLACK/HAITIAN                                                 101
ASIAN - ASIAN INDIAN                                           85


In [21]:
def map_etnic(x):
    x = str(x).strip().upper()
    if 'WHITE' in x:
        return 'WHITE'
    elif 'BLACK' in x or 'AFRICAN' in x:
        return 'BLACK'
    elif 'HISPANIC' in x or 'LATINO' in x:
        return 'HISPANIC'
    elif 'ASIAN' in x:
        return 'ASIAN'
    elif 'NATIVE' in x or 'PACIFIC ISLANDER' in x or 'ALASKA NATIVE' in x:
        return 'NATIVE/PACIFIC'
    elif 'PORTUGUESE' in x or 'MIDDLE EASTERN' in x or 'SOUTH AMERICAN' in x or 'CARIBBEAN' in x or 'OTHER' in x or 'MULTI' in x:
        return 'OTHER'
    elif 'UNKNOWN' in x or 'UNABLE' in x or 'DECLINED' in x:
        return 'UNKNOWN'
    else:
        return 'OTHER'

adm["ETHNIC_GROUP"] = adm["ETHNICITY"].apply(map_etnic)


In [23]:
adm["ETHNIC_GROUP"].value_counts(dropna = False)

ETHNIC_GROUP
WHITE             41325
UNKNOWN            5896
BLACK              5785
HISPANIC           2128
ASIAN              2007
OTHER              1763
NATIVE/PACIFIC       72
Name: count, dtype: int64

In [28]:
pd.crosstab(adm['ETHNIC_GROUP'],adm["HOSPITAL_EXPIRE_FLAG"])

HOSPITAL_EXPIRE_FLAG,0,1
ETHNIC_GROUP,,
ASIAN,1855,152
BLACK,5364,421
HISPANIC,2008,120
NATIVE/PACIFIC,64,8
OTHER,1627,136
UNKNOWN,4958,938
WHITE,37246,4079


In [30]:
del adm['ETHNICITY']


In [31]:
adm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58976 entries, 0 to 58975
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   SUBJECT_ID            58976 non-null  int64 
 1   HADM_ID               58976 non-null  int64 
 2   ADMITTIME             58976 non-null  object
 3   DISCHTIME             58976 non-null  object
 4   DEATHTIME             5854 non-null   object
 5   ADMISSION_TYPE        58976 non-null  object
 6   INSURANCE             58976 non-null  object
 7   MARITAL_STATUS        48848 non-null  object
 8   DIAGNOSIS             58951 non-null  object
 9   HOSPITAL_EXPIRE_FLAG  58976 non-null  int64 
 10  ETHNIC_GROUP          58976 non-null  object
dtypes: int64(3), object(8)
memory usage: 4.9+ MB


In [32]:
adm["ADMISSION_TYPE"].unique()

array(['EMERGENCY', 'ELECTIVE', 'NEWBORN', 'URGENT'], dtype=object)

In [40]:
pd.crosstab(adm['ADMISSION_TYPE'], adm["HOSPITAL_EXPIRE_FLAG"])

HOSPITAL_EXPIRE_FLAG,0,1
ADMISSION_TYPE,,
ELECTIVE,7509,197
EMERGENCY,36637,5434
NEWBORN,7801,62
URGENT,1175,161


In [41]:
ct = pd.crosstab(adm['ADMISSION_TYPE'], adm["HOSPITAL_EXPIRE_FLAG"])
ct_percent = ct.div(ct.sum(axis=1), axis=0) * 100
print(ct_percent)

HOSPITAL_EXPIRE_FLAG          0          1
ADMISSION_TYPE                            
ELECTIVE              97.443550   2.556450
EMERGENCY             87.083739  12.916261
NEWBORN               99.211497   0.788503
URGENT                87.949102  12.050898


In [43]:
adm['MARITAL_STATUS'].unique()

array(['MARRIED', 'SINGLE', nan, 'DIVORCED', 'WIDOWED', 'SEPARATED',
       'UNKNOWN (DEFAULT)', 'LIFE PARTNER'], dtype=object)

In [44]:
adm['MARITAL_STATUS'].value_counts()

MARITAL_STATUS
MARRIED              24239
SINGLE               13254
WIDOWED               7211
DIVORCED              3213
SEPARATED              571
UNKNOWN (DEFAULT)      345
LIFE PARTNER            15
Name: count, dtype: int64

In [48]:
ct_ms=pd.crosstab(adm['MARITAL_STATUS'],adm['HOSPITAL_EXPIRE_FLAG'])
ct_ms_percent = ct_ms.div(ct_ms.sum(axis=1), axis=0) * 100
print(ct_ms_percent)

HOSPITAL_EXPIRE_FLAG           0          1
MARITAL_STATUS                             
DIVORCED               90.071584   9.928416
LIFE PARTNER          100.000000   0.000000
MARRIED                89.232229  10.767771
SEPARATED              90.542907   9.457093
SINGLE                 91.760978   8.239022
UNKNOWN (DEFAULT)      81.159420  18.840580
WIDOWED                85.397310  14.602690


In [50]:
def map_marital_status(x):
    x = str(x).strip().upper()
    if x == 'MARRIED':
            return 'MARRIED'
    elif x == 'SINGLE':
        return 'SINGLE'
    elif x in ['DIVORCED','SEPARATED']:
        return 'DIVORCED/SEPARATED'
    elif x == 'WIDOWED':
        return 'WIDOWED'
    else:
        return 'UKNOWN/OTHER'
    
adm["MARITAL_STATUS"] = adm["MARITAL_STATUS"].apply(map_marital_status)
print(pd.crosstab(adm['MARITAL_STATUS'],adm['HOSPITAL_EXPIRE_FLAG']))

HOSPITAL_EXPIRE_FLAG      0     1
MARITAL_STATUS                   
DIVORCED/SEPARATED     3411   373
MARRIED               21629  2610
SINGLE                12162  1092
UKNOWN/OTHER           9762   726
WIDOWED                6158  1053


In [51]:
pd.crosstab(adm['MARITAL_STATUS'],adm['HOSPITAL_EXPIRE_FLAG'])

HOSPITAL_EXPIRE_FLAG,0,1
MARITAL_STATUS,,
DIVORCED/SEPARATED,3411,373
MARRIED,21629,2610
SINGLE,12162,1092
UKNOWN/OTHER,9762,726
WIDOWED,6158,1053


In [57]:
adm['DIAGNOSIS'].value_counts()

DIAGNOSIS
NEWBORN                                     7823
PNEUMONIA                                   1566
SEPSIS                                      1184
CONGESTIVE HEART FAILURE                     928
CORONARY ARTERY DISEASE                      840
                                            ... 
+ETT;DYSPNEA\CATH                              1
MYOCARDIAL INFARCTION/PNEUMONIA                1
TRACHEAL/BRONCHIAL ACHALSIA/SDA                1
BRAIN METASTESIS                               1
SHORTNESS OF BREATH;INCREASED CREATININE       1
Name: count, Length: 15691, dtype: int64

In [59]:
adm['INSURANCE'].value_counts()

INSURANCE
Medicare      28215
Private       22582
Medicaid       5785
Government     1783
Self Pay        611
Name: count, dtype: int64

In [60]:
pd.crosstab(adm['INSURANCE'],adm['HOSPITAL_EXPIRE_FLAG'])

HOSPITAL_EXPIRE_FLAG,0,1
INSURANCE,,
Government,1693,90
Medicaid,5404,381
Medicare,24310,3905
Private,21199,1383
Self Pay,516,95


In [61]:
ct_ins=pd.crosstab(adm['INSURANCE'],adm['HOSPITAL_EXPIRE_FLAG'])
ct_ins_percent = ct_ins.div(ct_ins.sum(axis=1), axis=0) * 100
print(ct_ins_percent)

HOSPITAL_EXPIRE_FLAG          0          1
INSURANCE                                 
Government            94.952328   5.047672
Medicaid              93.414002   6.585998
Medicare              86.159844  13.840156
Private               93.875653   6.124347
Self Pay              84.451718  15.548282


| Insurance Type | % Survived | % Died | Interpretation                                                                |
| -------------- | ---------- | ------ | ----------------------------------------------------------------------------- |
| **Government** | 94.95%     | 5.05%  | Low mortality rate — possibly younger or less severe cases                    |
| **Medicaid**   | 93.41%     | 6.59%  | Slightly higher mortality than Government insurance                           |
| **Medicare**   | 86.16%     | 13.84% | 🔴 Highest mortality — expected since Medicare mainly covers elderly patients |
| **Private**    | 93.88%     | 6.12%  | Similar to Medicaid — moderate mortality                                      |
| **Self Pay**   | 84.45%     | 15.55% | 🔴 Very high mortality — possibly late admission, lower access to care        |


In [79]:
adm['ADMITTIME'] = pd.to_datetime(adm['ADMITTIME'],errors='coerce')
adm['DISCHTIME'] = pd.to_datetime(adm['DISCHTIME'],errors='coerce')
adm['TIME_SPENT'] = adm['DISCHTIME'] - adm['ADMITTIME']                                                               

In [80]:
adm['TIME_SPENT'].head(5)

0   1 days 03:28:00
1   5 days 11:55:00
2   6 days 18:26:00
3   2 days 20:34:00
4   3 days 12:49:00
Name: TIME_SPENT, dtype: timedelta64[ns]

In [82]:
pd.crosstab(adm['TIME_SPENT'],adm['HOSPITAL_EXPIRE_FLAG'])

HOSPITAL_EXPIRE_FLAG,0,1
TIME_SPENT,,
-1 days +01:19:00,0,1
-1 days +03:45:00,0,1
-1 days +04:52:00,0,1
-1 days +05:33:00,0,1
-1 days +06:01:00,0,1
...,...,...
181 days 21:25:00,1,0
191 days 10:09:00,0,1
202 days 18:18:00,1,0


In [92]:
# Boolean mask for negative timedeltas
neg_time_mask = adm["TIME_SPENT"] < pd.Timedelta(0)

# Number of rows with negative time spent
num_neg_rows = neg_time_mask.sum()
print(num_neg_rows)


0


In [90]:
adm[neg_time_mask].tail(5)


,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,INSURANCE,MARITAL_STATUS,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,ETHNIC_GROUP,TIME_SPENT
57233,96810,165589,2103-09-25 20:38:00,2103-09-25 12:00:00,2103-09-25 12:00:00,EMERGENCY,Medicaid,WIDOWED,CARDIAC ARREST,1,WHITE,-1 days +15:22:00
57559,98943,193747,2164-11-14 20:03:00,2164-11-14 12:00:00,2164-11-14 12:00:00,EMERGENCY,Medicare,MARRIED,INTRACRANIAL HEMORRHAGE,1,WHITE,-1 days +15:57:00
57902,99207,191004,2143-07-06 19:59:00,2143-07-06 12:00:00,2143-07-06 12:00:00,EMERGENCY,Medicaid,DIVORCED/SEPARATED,GSW L. CHEST WALL & ABDOMEN,1,WHITE,-1 days +16:01:00
58346,90642,101946,2122-04-24 14:36:00,2122-04-24 12:00:00,2122-04-24 12:00:00,EMERGENCY,Private,MARRIED,ST-SEGMENT ELEVATION MYOCARDIAL INFARCTION\CATH,1,UNKNOWN,-1 days +21:24:00
58884,95367,139266,2135-04-03 14:16:00,2135-04-03 12:00:00,2135-04-03 12:00:00,EMERGENCY,Medicare,MARRIED,CHEST PAIN,1,WHITE,-1 days +21:44:00


In [91]:
# Keep only rows where TIME_SPENT >= 0
adm = adm[adm["TIME_SPENT"] >= pd.Timedelta(0)]
adm.reset_index(drop=True, inplace=True)


In [5]:
adm['TIME_SPENT'].isna().sum() == 0
(adm['DISCHTIME'] > adm['ADMITTIME']).all()


np.True_

In [94]:
adm.to_csv("cleaned_admission.csv")

In [6]:
adm['SUBJECT_ID'].duplicated().sum()

np.int64(12433)

In [8]:
adm['SUBJECT_ID'].value_counts()

SUBJECT_ID
13033    42
109      34
11861    34
5060     31
20643    24
         ..
27        1
26        1
25        1
24        1
22        1
Name: count, Length: 46445, dtype: int64